In [5]:
import tabula 
import pandas as pd
from tika import parser
import os
import copy
import urllib

In [6]:
def parsing_table(file, df_all1, df_all2, df_all3, dai):  
#     if not os.path.isfile(file):
    try:
        raw = parser.from_file(file)
#     except urllib.error.HTTPError:
    except FileNotFoundError:
        skipped.append(dai)
        return df_all1, df_all2, df_all3
    if raw['status'] == 200 and raw['content'] is not None and len(raw['content'].strip().split()) > 200 and raw['content'].strip().split()[4] == '№' and raw['content'].strip().split()[5][0] == 'D':
        df = tabula.read_pdf(file, pages=1, spreadsheet=True, pandas_options={'header': None}, multiple_tables=True)
        if type(df) == list and len(df) == 2:
            df[0]['decl_num'] = raw['content'].strip().split()[5]
            df[1]['decl_num'] = raw['content'].strip().split()[5]
        else:
            print('akusi', dai)
        if df_all1 is None:
            df_all1 = df[0]
            df_all2 = df[1]
        else:
            df_all1 = pd.concat([df_all1, df[0]], ignore_index=True)
            df_all2 = pd.concat([df_all2, df[1]], ignore_index=True)
        df = tabula.read_pdf(file, pages=2, spreadsheet=True, pandas_options={'header': None}, multiple_tables=True)
        if type(df) == list and len(df) >= 1:
            df = [j for j in df if type(j) == pd.core.frame.DataFrame and j.shape != (1, 1)][0]
        for i in range(3, int(raw['metadata']['xmpTPg:NPages']) + 1):
            f2 = tabula.read_pdf(file,
                         pages=i, pandas_options={'header': None}, spreadsheet=True, multiple_tables=True)
            if type(f2) == list and len(f2) >= 1:
                f2 = [j for j in f2 if type(j) == pd.core.frame.DataFrame and j.shape != (1, 1)][0]
            if type(f2) != list:
                df = pd.concat([df, f2], ignore_index=True)
        df['decl_num'] = raw['content'].strip().split()[5]
        if df_all3 is None:
            df_all3 = df
        else:
            df_all3 = pd.concat([df_all3, df], ignore_index=True)
    return df_all1, df_all2, df_all3

In [7]:
def data_cleaning(dat1, dat2, dat3):
#     print(path_to, file_num, 2)
    dat3 = dat3.loc[(dat3[2].isnull() == False) & (dat3[4].isnull() == False) & (dat3[2] != '3' ) & (dat3[4] != 'Клас\rнебезпеки')]
    for i in range(12):
        dat3[i] = dat3[i].apply(lambda x: str(x).replace('\r', ' '))
    dat3.columns = ['No','trash_name','code','alter_name','class','storage','value',
                                 'value_prediction','destination','value_destination','operation','operation_desc', 'decl_num']

    dat1[0] = dat1[0].apply(lambda x: str(x).replace('\r', ' '))
    dat1[1] = dat1[1].apply(lambda x: str(x).replace('\r', ' '))
    
    print(dat2[dat2[0].isnull()][dat2[dat2[0].isnull()][5] != '(x1)'], "AAAAAAAAAAAAAAAAAAAAAAAAAAAAA")
    for i in range(6):
        dat2[i] = dat2[i].apply(lambda x: str(x).replace('\r', ' '))
    dat2 = dat2.loc[(dat2[0] != 'Період утворення відходів') & (dat2[5] != '(x1)')]
    dat2.columns = ['waste_creation_time','index_gen_wastes','amount_I','amount_II', 'amount_III', 'amount_IV', 'decl_num']
    return dat1, dat2, dat3

In [8]:
def data_saving(datas, file_num):
    datas[0], datas[1], datas[2] = data_cleaning(datas[0], datas[1], datas[2])
    for j in range(3):
        datas[j].to_csv(path_to + files[j] + str(file_num) + ".csv", sep=';', index=False)
#     file_num += 1000
#         if not os.path.isfile(path_to + files[j]):
#             datas[j].to_csv(path_to + files[j], sep=';', index=False)
#         else:
#             with open(path_to + files[j], 'a') as f:
#                 datas[j].to_csv(path_to + files[j], sep=';', index=False, header=False)
    return [None, None, None]

In [9]:
datas = [None, None, None]
# path_from = 'https://e-eco.gov.ua/parlor/applications/viewfile/id/' # in case we download on flight
path_from = './declarations/' # where are downloaded declarations
path_to = "./dec/data/d2/" # where we save files
file_num = 41000 # file_num - 999 = starting file of our work => min file_num == 1000, and file_num % 1000 == 0
files = ["F", "S", "T"]
skipped = []
min_range = file_num - 999
max_range = 42001 # max range of files to 

for i in range(min_range, max_range):
    if i % 100 == 0:
        print('--', i)
    datas[0], datas[1], datas[2] = parsing_table(path_from + str(i), datas[0], datas[1], datas[2], i)
    if i % 1000 == 0:
        datas = data_saving(datas, file_num)
        file_num += 1000
        
datas = data_saving(datas, 0)

2019-05-02 14:29:59,911 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.19/tika-server-1.19.jar to /tmp/tika-server.jar.
2019-05-02 14:30:51,503 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.19/tika-server-1.19.jar.md5 to /tmp/tika-server.jar.md5.
2019-05-02 14:30:52,218 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


-- 40100
-- 40200
-- 40300
-- 40400
-- 40500
-- 40600
-- 40700
-- 40800
-- 40900
-- 41000


/home/roman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, decl_num]
Index: [] AAAAAAAAAAAAAAAAAAAAAAAAAAAAA
-- 41100
-- 41200
-- 41300
-- 41400
-- 41500
-- 41600
-- 41700
-- 41800
-- 41900
-- 42000


/home/roman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, decl_num]
Index: [] AAAAAAAAAAAAAAAAAAAAAAAAAAAAA


AttributeError: 'NoneType' object has no attribute 'loc'

In [11]:
print(pd.read_csv("./dec/data/fulls/Declarations_first_table.csv", sep=';').shape)
print(pd.read_csv("./dec/data/fulls/Declarations_second_table.csv", sep=';').shape)
print(pd.read_csv("./dec/data/fulls/Declarations_third_table.csv", sep=';').shape)

(108540, 3)
(43416, 7)
(189335, 13)


/home/roman/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
#join files into one csv file
for f in files:
    data1 = pd.read_csv("./dec/data/d2/" + f + "1000.csv", sep=';')
    for i in range(2000,43000,1000):
        data2 = pd.read_csv("./dec/data/d2/" + f + str(i) + ".csv", sep=';')
        data1 = pd.concat([data1,data2], ignore_index=True)
    if f == 'F':
        data1.to_csv("./dec/data/fulls/Declarations_first_table.csv", sep=';', index=False)
    elif f == 'S':
        data1.to_csv("./dec/data/fulls/Declarations_second_table.csv", sep=';', index=False)
    elif f == 'T':
        data1.to_csv("./dec/data/fulls/Declarations_third_table.csv", sep=';', index=False)

In [13]:
print(pd.read_csv("./dec/data/fulls/Declarations_first_table.csv", sep=';').shape)
print(pd.read_csv("./dec/data/fulls/Declarations_second_table.csv", sep=';').shape)
print(pd.read_csv("./dec/data/fulls/Declarations_third_table.csv", sep=';').shape)

(109630, 3)
(43852, 7)
(190915, 13)
